In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D


/home/ivm/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:

def create_model():


	num_classes = 10
	model = Sequential()
	input_shape = (28, 28)
	model.add(Flatten(input_shape=input_shape))
	model.add(Dense(28*28*10, activation="tanh"))
	model.add(Dense(1000, activation="tanh"))
	model.add(Dense(num_classes, activation="softmax"))
	opt = keras.optimizers.SGD(lr=0.001, decay=1e-6)

	model.compile(loss='categorical_crossentropy',
		      optimizer=opt,
		      metrics=['accuracy'])

	return model



In [3]:
def create_model():


    num_classes = 10
    model = Sequential()
    input_shape = (28, 28)
    model.add(Reshape((28,28,1), input_shape=input_shape))
    model.add(Conv2D(10, (5,5)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(20, (3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(40, (3,3)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(100, activation="tanh"))
    model.add(Dense(num_classes, activation="softmax"))
    opt = keras.optimizers.SGD(lr=1.0, decay=1e-6)

    model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

    return model

model = create_model()
model.summary()


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 10)        260       
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 10)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 10)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 20)        1820      
_________________________________________________________________
activation_2 (Activation)    (None, 10, 10, 20)        0         
_________________________________________________________________
max_

In [4]:
#__worker_class__

from striped.ml.keras_backend import ML_Keras_Worker
from striped.ml import ML_Accumulator

class Worker(ML_Keras_Worker):

        def __init__(self, params, bulk, job_interface, db_interface):
            ML_Keras_Worker.__init__(self, params, bulk, "image", "labels")

class Accumulator(ML_Accumulator):
        pass



In [7]:
from striped.job import Session
from striped.ml import ML_Job, MomentumOptimizer


session = Session("striped_130tb.yaml")

for epoch in range(5):
    job = ML_Job(session, model, worker_file="worker.py", optimizer=MomentumOptimizer(0.1, 0.5))
    job.run("MNIST")
    print "epoch: %d, runtime: %f, loss: %s, accuracy: %.1f%%" % (epoch+1, job.Runtime, job.Loss, job.Metric*100)


epoch: 1, runtime: 64.585906, loss: 2.32160348495, accuracy: 3.9%
epoch: 2, runtime: 76.023883, loss: 2.29838680824, accuracy: 7.4%
epoch: 3, runtime: 75.638259, loss: 2.27433430354, accuracy: 13.2%
epoch: 4, runtime: 80.241803, loss: 2.2465686957, accuracy: 19.5%
epoch: 5, runtime: 91.992950, loss: 2.21221899986, accuracy: 26.5%
